<a href="https://colab.research.google.com/github/burnedsyn/enablecudaopencvcolab/blob/main/PRJ_VIDEO_mesh_cr%C3%A9ation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/opencv/opencv
!git clone https://github.com/opencv/opencv_contrib
!mkdir /content/build
%cd /content/build

!cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules  \
       -DBUILD_SHARED_LIBS=OFF \
       -DBUILD_TESTS=OFF \
       -DBUILD_PERF_TESTS=OFF \
       -DBUILD_EXAMPLES=OFF \
       -DWITH_OPENEXR=OFF \
       -DWITH_CUDA=ON \
       -DWITH_CUBLAS=ON \
       -DWITH_CUDNN=ON \
       -DOPENCV_DNN_CUDA=ON \
       /content/opencv

!make -j8

In [ ]:
# connect your Google Drive
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


In [ ]:
!pip install opencv-python-headless mediapipe numpy

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os

cap = cv2.VideoCapture('/content/gdrive/MyDrive/AlicePRJ/alicvideo.mp4')

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Create output directories
output_dir = "/content/gdrive/MyDrive/AlicePRJ/"
frame_dir = os.path.join(output_dir, "frames")
landmarks_dir = os.path.join(output_dir, "landmarks")
landmarks_only_dir = os.path.join(output_dir, "landmarks_only")
csv_dir = os.path.join(output_dir, "csv")
os.makedirs(frame_dir, exist_ok=True)
os.makedirs(landmarks_dir, exist_ok=True)
os.makedirs(landmarks_only_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)

# Create CSV file to store landmarks and coordinates
csv_file = open(os.path.join(csv_dir, "landmarks.csv"), "w")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Frame", "Landmark", "X", "Y", "Z"])

frame_num = 0

while cap.isOpened():
    # Read frame from video
    ret, frame = cap.read()
    if not ret:
        break
        
    # Flip the frame horizontally for selfie view display
    frame = cv2.flip(frame, 1)
    
    # Convert the image from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # To improve performance, optionally mark the image as not writeable to pass by reference.
    image.flags.writeable = False
    
    # Process the image with the Mediapipe Mesh solution
    results = face_mesh.process(image)
    
    # Draw the face mesh on the frame
    if results.multi_face_landmarks:
        csv_file_path = os.path.join(csv_dir, f'frame_{frame_num}.csv')
        for face_landmarks in results.multi_face_landmarks:
              # Save the landmark coordinates to a CSV file
           
            with open(csv_file_path, 'w') as csv_file:
                csv_writer.writerow(["Frame", "Landmark", "X", "Y", "Z"])
                for landmark in face_landmarks.landmark:
                    csv_file.write(f'{landmark.x},{landmark.y},{landmark.z}\n')
    
                
            # Draw the face mesh on the frame
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)
            
            # Save the landmark and the landmark only in separate image files
            landmarks_filename = f"landmarks_{frame_num:04d}.jpg"
            landmarks_path = os.path.join(landmarks_dir, landmarks_filename)
            cv2.imwrite(landmarks_path, frame)
            
            landmarks_only = np.zeros_like(image)
            mp_drawing.draw_landmarks(landmarks_only, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)
            landmarks_only_filename = f"landmarks_only_{frame_num:04d}.jpg"
            landmarks_only_path = os.path.join(landmarks_only_dir, landmarks_only_filename)
            cv2.imwrite(landmarks_only_path, landmarks_only)
            
    # Save each frame as an image file
    frame_filename = f"frame_{frame_num:04d}.jpg"
    frame_path = os.path.join(frame_dir, frame_filename)
    cv2.imwrite(frame_path, image)
    
    
    
    # Increment the frame number
    frame_num += 1

    # Exit the program when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

csv_file.close()
cap.release()
cv2.destroyAllWindows()


In [ ]:
!ffmpeg -framerate 30 -pattern_type glob -i '/content/gdrive/MyDrive/AlicePRJ/frames/*.jpg' -c:v libx264 -pix_fmt yuv420p /content/gdrive/MyDrive/AlicePRJ/frames/reconstructoriginal.mp4
!ffmpeg -framerate 30 -pattern_type glob -i '/content/gdrive/MyDrive/AlicePRJ/landmarks/*.jpg' -c:v libx264 -pix_fmt yuv420p /content/gdrive/MyDrive/AlicePRJ/landmarks/reconstructlandmark.mp4
!ffmpeg -framerate 30 -pattern_type glob -i '/content/gdrive/MyDrive/AlicePRJ/landmarks_only/*.jpg' -c:v libx264 -pix_fmt yuv420p /content/gdrive/MyDrive/AlicePRJ/landmarks_only/reconstructlandmarks_only.mp4



ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:

import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture('/content/gdrive/MyDrive/AlicePRJ/alicvideo.mp4')

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

while cap.isOpened():
    # Read frame from video
    ret, frame = cap.read()
    if not ret:
        break
        
    # Flip the frame horizontally for selfie view display
    frame = cv2.flip(frame, 1)
    
    # Convert the image from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # To improve performance, optionally mark the image as not writeable to pass by reference.
    image.flags.writeable = False
    
    # Process the image with the Mediapipe Mesh solution
    results = face_mesh.process(image)
    
    # Draw the face mesh on the frame
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)
            
    
    # Show the frame
    cv2_imshow(frame)
    
    
    # Exit the program when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
!pip install opencv-python-headless mediapipe numpy

Accelerateur gpu

In [ ]:
import cv2
import mediapipe as mp
import tensorflow as tf

# Enable TensorFlow GPU support
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Using GPU: ", tf.test.gpu_device_name())
    except RuntimeError as e:
        print(e)

# Create the Face Mesh model
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Create a VideoCapture object to read the video file
cap = cv2.VideoCapture('/content/gdrive/MyDrive/AlicePRJ/alicvideo.mp4')


# Read the first frame to get the frame size
ret, frame = cap.read()
frame_height, frame_width, _ = frame.shape

# Create a CUDA stream for GPU-accelerated OpenCV functions
cuda_stream = cv2.cuda_Stream()

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create CUDA matrices for GPU-accelerated OpenCV functions
    cuda_image = cv2.cuda_GpuMat(image, allocator=cv2.cuda.HostMemAllocator())
    cuda_gray = cv2.cuda_GpuMat((frame_height, frame_width), cv2.CV_8UC1, allocator=cv2.cuda.HostMemAllocator())
    cuda_threshold = cv2.cuda_GpuMat((frame_height, frame_width), cv2.CV_8UC1, allocator=cv2.cuda.HostMemAllocator())

    # Convert the image to grayscale on the GPU
    cv2.cuda.cvtColor(cuda_image, cv2.COLOR_RGB2GRAY, dst=cuda_gray, stream=cuda_stream)

    # Threshold the grayscale image on the GPU
    cv2.cuda.threshold(cuda_gray, 127, 255, cv2.THRESH_BINARY, dst=cuda_threshold, stream=cuda_stream)

# Download the thresholded image back to the CPU
thresholded_image = cuda_threshold.download()

# Process the thresholded image with the Face Mesh model
results = face_mesh.process(thresholded_image)

# Draw the face mesh on the frame
if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

# Display the frame
cv2.imshow('Frame', frame)

# Wait for a key press
if cv2.waitKey(1) & 0xFF == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()
